In [ ]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import HTML

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)

import settings
import helper
import visual

# 1. Load Meta File and Annotation

In [ ]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

In [ ]:
df_meta_train.head(5)

In [ ]:
visual.plot_meta(df_meta_train, title='meta_train')

In [ ]:
df_meta_test.head(5)

In [ ]:
visual.plot_meta(df_meta_test, title='meta_test')

In [ ]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

In [ ]:
df_annotation.sample(10)

In [ ]:
visual.plot_annotation(df_annotation, title='annotation')

In [ ]:
dict_label = { '结节': 0, '索条': 0, '动脉硬化或钙化': 0, '淋巴结钙化': 0 }
for i,item in df_annotation.iterrows():
    if item.label == 1.:
        dict_label['结节'] += 1
    elif item.label == 5.:
        dict_label['索条'] += 1
    elif item.label == 31.:
        dict_label['动脉硬化或钙化'] += 1
    elif item.label == 32.:
        dict_label['淋巴结钙化'] += 1
        
print(dict_label)

In [ ]:
visual.plot_annotation(df_annotation[(df_annotation['label'] == 1.) |  (df_annotation['label'] == 5.)], title='annotation')

In [ ]:
visual.plot_annotation(df_annotation[(df_annotation['label'] == 31.) |  (df_annotation['label'] == 32.)], title='annotation')

# 2. Load Lung as Array from disk

In [ ]:
EXAMPLE_SERIESUID = '542588'
DATASET = 'TRAIN'

In [ ]:
df = df_meta_train if DATASET == 'TRAIN' else df_meta_test
record = df.loc[EXAMPLE_SERIESUID]
print(type(record), record)

In [ ]:
lung_l, mask_l = helper.load_lung_array(EXAMPLE_SERIESUID, int(record.width), int(record.height), int(record.slice), wtype='lung')
lung_m, mask_m = helper.load_lung_array(EXAMPLE_SERIESUID, int(record.width), int(record.height), int(record.slice), wtype='medi')


# 3. Visualize 2d and 3d Lung

In [ ]:
visual.plot_slices(lung_l, title=f'{EXAMPLE_SERIESUID} [lung][{DATASET}]')

In [ ]:
visual.plot_slices(lung_l*mask_l, title=f'{EXAMPLE_SERIESUID} [lung][{DATASET}][MASK]')

In [ ]:
visual.plot_slices(lung_m, title=f'{EXAMPLE_SERIESUID} [medi][{DATASET}]')

In [ ]:
visual.plot_slices(lung_m*mask_m, title=f'{EXAMPLE_SERIESUID} [medi][{DATASET}][MASK]')

In [ ]:
# anim = visual.anim_slices(lung_l, title=f'{EXAMPLE_SERIESUID} [{DATASET}]')
# HTML(anim.to_html5_video())

In [ ]:
# anim = visual.anim_slices(lung_l*mask_l, title=f'{EXAMPLE_SERIESUID} [{DATASET}]')
# HTML(anim.to_html5_video())

In [ ]:
# z,y,x -> y,z,x front view
# anim = visual.anim_slices(lung_l.transpose(1,0,2), title=f'{EXAMPLE_SERIESUID} [{DATASET}]')
# HTML(anim.to_html5_video())

In [ ]:
# z,y,x -> x,z,y side view
# anim = visual.anim_slices(lung_l.transpose(2,0,1), title=f'{EXAMPLE_SERIESUID} [{DATASET}]')
# HTML(anim.to_html5_video())

In [ ]:
# visual.plot_3d_lung(lung_l, title=f'{EXAMPLE_SERIESUID} [{DATASET}]')

In [ ]:
# visual.plot_3d_lung(lung_l*mask_l, title=f'{EXAMPLE_SERIESUID} [{DATASET}]')

# 4.1 Visualize Annotation Label 31~32

In [ ]:
EXAMPLE_SERIESUID = '542588'
DATASET = 'TRAIN'

In [ ]:
df = df_meta_train if DATASET == 'TRAIN' else df_meta_test
record = df.loc[EXAMPLE_SERIESUID]
print(type(record), record)

In [ ]:
lung, mask = helper.load_lung_array(EXAMPLE_SERIESUID, int(record.width), int(record.height), int(record.slice), wtype='medi')

In [ ]:
labels = df_annotation.loc[[EXAMPLE_SERIESUID]]
labels = labels[(labels['label']==31.)|(labels['label']==32.)]

In [ ]:
visual.plot_labels(lung, mask, labels, title=f'{EXAMPLE_SERIESUID} [LABELS:{len(labels)}]')

# 4.2 Visualize Annotation Label 1,5

In [ ]:
EXAMPLE_SERIESUID = '542588'
DATASET = 'TRAIN'

In [ ]:
df = df_meta_train if DATASET == 'TRAIN' else df_meta_test
record = df.loc[EXAMPLE_SERIESUID]
print(type(record), record)

In [ ]:
lung, mask = helper.load_lung_array(EXAMPLE_SERIESUID, int(record.width), int(record.height), int(record.slice), wtype='lung')

In [ ]:
labels = df_annotation.loc[[EXAMPLE_SERIESUID]]
labels = labels[(labels['label']==1.)|(labels['label']==5.)]

In [ ]:
visual.plot_labels(lung, mask, labels, title=f'{EXAMPLE_SERIESUID} [LABELS:{len(labels)}]')

# 5. Exception Cases Analytics

In [ ]:
df_anno_ex = df_annotation[(df_annotation['diameterX'] > 80)|(df_annotation['diameterY'] > 100)|(df_annotation['diameterZ'] > 80)]
df_anno_ex.shape

In [ ]:
for lung_id in set(df_anno_ex.index):
    record = df_meta_train.loc[lung_id]
    lung_l, mask_l = helper.load_lung_array(lung_id, int(record.width), int(record.height), int(record.slice), wtype='lung')
    lung_m, mask_m = helper.load_lung_array(lung_id, int(record.width), int(record.height), int(record.slice), wtype='medi')
    labels = df_annotation.loc[[lung_id]]
    
    if 1. in set(labels['label']) or 5. in set(labels['label']):
        visual.plot_labels(lung_l, mask_l, labels, title=f'{lung_id} [LABELS:{len(labels)}]')
    elif 31. in set(labels['label']) or 32. in set(labels['label']):
        visual.plot_labels(lung_m, mask_m, labels, title=f'{lung_id} [LABELS:{len(labels)}]')
    